# Final Project: Counties Data Wrangling

- **Vintage**:  2020
- **Geography Level**: County   
- **Variables**:  https://api.census.gov/data/2020/acs/acs5/profile/variables.html 
- **Supported Geographies**: https://api.census.gov/data/2020/acs/acs5/profile/geography.html

### ***Question***:  What is the estimation and percent of population who speak Spanish at home for each county in California?  

## 1. Import necessary packages

In [187]:
import pandas as pd
import json
import requests

## 2. Build the API Request URL

### 2.1. Base URL

In [188]:
base_url = "https://api.census.gov/data"

### 2.2. Dataset Name

In [189]:
dataset_name = "/2020/acs/acs5/profile"

### 2.3. Get Variables

- **DP02_0116E**: Estimate of population (5 years and over) who speaks Spanish at home
- **DP02_0116PE**: Percent of population (5 years and over) who speaks Spanish at home

In [190]:
get_variables = "?get=NAME,DP02_0116E,DP02_0116PE"

### 2.4. Geography Levels 

- Every state in the US

In [191]:
geography = "&for=county:*&in=state:06"

### 2.5. Put it all together 

In [192]:
request_url = base_url + dataset_name + get_variables + geography
print("request_url = ", request_url)

request_url =  https://api.census.gov/data/2020/acs/acs5/profile?get=NAME,DP02_0116E,DP02_0116PE&for=county:*&in=state:06


## 3. Make the API call

In [193]:
# Make API Call
r = requests.get(request_url)

api_results = r.json()

In [194]:
type(api_results)

list

## 4. Get the data into a Dataframe 

In [195]:
data = pd.DataFrame(api_results)

data.head()

,0,1,2,3,4
0,NAME,DP02_0116E,DP02_0116PE,state,county
1,"Alameda County, California",250597,16.0,06,001
2,"Alpine County, California",110,10.1,06,003
3,"Butte County, California",21187,10.0,06,007
4,"Colusa County, California",9998,50.2,06,011


## 5. Get the first Row into columns and then get rid of it

In [196]:
data.columns = data.iloc[0]

data = data.iloc[1:]

print("Number of rows:", data.shape[0])
print("Number of columns:", data.shape[1])
data.head()

Number of rows: 58
Number of columns: 5


,NAME,DP02_0116E,DP02_0116PE,state,county
1,"Alameda County, California",250597,16.0,06,001
2,"Alpine County, California",110,10.1,06,003
3,"Butte County, California",21187,10.0,06,007
4,"Colusa County, California",9998,50.2,06,011
5,"Contra Costa County, California",195737,18.1,06,013


## 6. Add Rural areas

### 6.1 Import csv with states abbreviations

In [197]:
rural_areas = pd.read_excel('Data/Rural_areas_California.xlsx', dtype={'Rural_Status' : str})

print("Number of rows:", rural_areas.shape[0])
print("Number of columns:", rural_areas.shape[1])
rural_areas.head()

Number of rows: 58
Number of columns: 4


,FIPS_Code,Abbreviation,County_Name,Rural_Status
0,6001,CA,Alameda County,1
1,6003,CA,Alpine County,4
2,6005,CA,Amador County,4
3,6007,CA,Butte County,2
4,6009,CA,Calaveras County,6


### 6.2. Splitting the NAME column in the dataframe data for merging both both dataframes

In [198]:
two_new_cols = ['County_Name', 'State_Name']

data[two_new_cols] = data['NAME'].str.split(', ',1, expand=True)

print(data.shape)
data.head()

(58, 7)


,NAME,DP02_0116E,DP02_0116PE,state,county,County_Name,State_Name
1,"Alameda County, California",250597,16.0,06,001,Alameda County,California
2,"Alpine County, California",110,10.1,06,003,Alpine County,California
3,"Butte County, California",21187,10.0,06,007,Butte County,California
4,"Colusa County, California",9998,50.2,06,011,Colusa County,California
5,"Contra Costa County, California",195737,18.1,06,013,Contra Costa County,California


### 6.3. Merge both dataframes (df and state_abb)

In [199]:
left_table = data
right_table = rural_areas

In [200]:
left_table_join_field = 'County_Name'
right_table_join_field = 'County_Name'

In [201]:
df = pd.merge(left_table,       
                right_table,     
                left_on=left_table_join_field,
                right_on=right_table_join_field,
                how='left'                          # Type of Join:  Left
            )

print()
print("Left Table:  ", left_table.shape)
print("Right Table: ", right_table.shape)
print("Joined Dataframe: ", df.shape)
print()

df.head()


Left Table:   (58, 7)
Right Table:  (58, 4)
Joined Dataframe:  (58, 10)



,NAME,DP02_0116E,DP02_0116PE,state,county,County_Name,State_Name,FIPS_Code,Abbreviation,Rural_Status
0,"Alameda County, California",250597,16.0,06,001,Alameda County,California,6001,CA,1
1,"Alpine County, California",110,10.1,06,003,Alpine County,California,6003,CA,4
2,"Butte County, California",21187,10.0,06,007,Butte County,California,6007,CA,2
3,"Colusa County, California",9998,50.2,06,011,Colusa County,California,6011,CA,4
4,"Contra Costa County, California",195737,18.1,06,013,Contra Costa County,California,6013,CA,1


## 7. Cleaning

### 7.1. Dropping columns

In [202]:
drop_columns = ['NAME', 'FIPS_Code']
df.drop(drop_columns, axis='columns', inplace=True)

print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])
df.head()

Number of rows: 58
Number of columns: 8


,DP02_0116E,DP02_0116PE,state,county,County_Name,State_Name,Abbreviation,Rural_Status
0,250597,16.0,06,001,Alameda County,California,CA,1
1,110,10.1,06,003,Alpine County,California,CA,4
2,21187,10.0,06,007,Butte County,California,CA,2
3,9998,50.2,06,011,Colusa County,California,CA,4
4,195737,18.1,06,013,Contra Costa County,California,CA,1


### 7.2. Renaming columns

In [203]:
cols_to_rename = {
                   'DP02_0116E' : 'Language spoken at home (Spanish) (DP02_0116E)', 
                   'DP02_0116PE' : 'Language spoken at home (Spanish) - Percent (DP02_0116PE)', 
                   'state' : 'FIPS_State', 
                   'county' : 'FIPS_County'
                 }
df.rename(columns = cols_to_rename, inplace=True)

print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])
df.head()

Number of rows: 58
Number of columns: 8


,Language spoken at home (Spanish) (DP02_0116E),Language spoken at home (Spanish) - Percent (DP02_0116PE),FIPS_State,FIPS_County,County_Name,State_Name,Abbreviation,Rural_Status
0,250597,16.0,06,001,Alameda County,California,CA,1
1,110,10.1,06,003,Alpine County,California,CA,4
2,21187,10.0,06,007,Butte County,California,CA,2
3,9998,50.2,06,011,Colusa County,California,CA,4
4,195737,18.1,06,013,Contra Costa County,California,CA,1


### 7.3. Reordering columns

In [204]:
cols_to_keep = ['County_Name', 'FIPS_State', 'FIPS_County', 'Rural_Status', 'Language spoken at home (Spanish) (DP02_0116E)', 'Language spoken at home (Spanish) - Percent (DP02_0116PE)', 'State_Name', 'Abbreviation']
df = df[cols_to_keep]

print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])
df.head()

Number of rows: 58
Number of columns: 8


,County_Name,FIPS_State,FIPS_County,Rural_Status,Language spoken at home (Spanish) (DP02_0116E),Language spoken at home (Spanish) - Percent (DP02_0116PE),State_Name,Abbreviation
0,Alameda County,06,001,1,250597,16.0,California,CA
1,Alpine County,06,003,4,110,10.1,California,CA
2,Butte County,06,007,2,21187,10.0,California,CA
3,Colusa County,06,011,4,9998,50.2,California,CA
4,Contra Costa County,06,013,1,195737,18.1,California,CA


### 7.4. Convert Rural_Status column

In [205]:
# Urban areas
urban_numbers = ['1','2']
for num in urban_numbers:
    df['Rural_Status'] = df['Rural_Status'].str.replace(num,'Urban')

# Rural areas
rural_numbers = ['3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
for num in rural_numbers:
    df['Rural_Status'] = df['Rural_Status'].str.replace(num,'Rural')

# Print
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])
df.head()

Number of rows: 58
Number of columns: 8


,County_Name,FIPS_State,FIPS_County,Rural_Status,Language spoken at home (Spanish) (DP02_0116E),Language spoken at home (Spanish) - Percent (DP02_0116PE),State_Name,Abbreviation
0,Alameda County,06,001,Urban,250597,16.0,California,CA
1,Alpine County,06,003,Rural,110,10.1,California,CA
2,Butte County,06,007,Urban,21187,10.0,California,CA
3,Colusa County,06,011,Rural,9998,50.2,California,CA
4,Contra Costa County,06,013,Urban,195737,18.1,California,CA


## 8. Save the Dataframe as a CSV file

In [206]:
csv_file_to_create = "Counties_Data.csv"

filename_with_path = "Data/" + csv_file_to_create
df.to_csv(filename_with_path, index=False)